http://web.stanford.edu/class/cs224n/assignment1/assignment1_soln.pdf

# (a) 

The sigmoid function is identical to logistic function. I've derived its graident multiple times. Here it is again.

$$\sigma = \frac{1}{1 + e^{-x}} = (1 + e^{-x})^{-1}$$

Then,
\begin{align*}
\frac{d \sigma}{dx} &= - (1 + e^{-x})^{-2} e^{-x} (-1) \\
                                 &=\frac{e^{-x}}{(1 + e^{-x})^2} \\
                                 &=(\frac{1}{1 + e^{-x}}) (\frac{e^{-x}}{1 + e^{-x}}) \\
                                 &=\sigma(1 - \sigma)
\end{align*}

# (b)

Given

\begin{align*}
\hat y = \mathrm{softmax}(\theta) = \begin{bmatrix}
e^{\theta_1} / \sum_j e^{\theta_j} \\ 
e^{\theta_2} / \sum_j e^{\theta_j} \\ 
\vdots \\ 
e^{\theta_q} / \sum_j e^{\theta_j} \\ 
\end{bmatrix}
\end{align*}

suppose there are $q$ classes, i.e $\hat y$ is a $q$-dimension vector.

$$
CE(y, \hat y) = J(y, \hat y) = 1\{y_i = 1\} y_i \mathrm{log}(\hat y_i)
$$

\begin{align*}
\frac{\partial J}{\partial \theta_j} 
&= \sum_{i}^{q} y_i \frac{1}{\hat y_i} \frac{\partial \hat y_i}{\partial \theta_j} \\ 
&= 1\{y_i = 1\} \frac{\sum_k e^{-\theta_k}}{e^{-\theta_i}} \frac{\partial \frac{e^{-\theta_i}}{\sum_{k} e^{-\theta_k}}}{\partial \theta_j} \\ 
&= 1\{y_i = 1\} \frac{\sum_k e^{-\theta_k}}{e^{-\theta_i}} \frac{\frac{\partial e^{-\theta_i}}{\partial \theta_j} \sum_k e^{-\theta_k} - e^{-\theta_i} \frac{\partial \sum_k e^{-\theta_k}}{\partial \theta_j}}{(\sum_k e^{-\theta_k})^2} \\ 
&= 1\{y_i = 1\} \frac{1}{e^{-\theta_i}} \frac{1\{i = j \} (- e^{-\theta_i}  \sum_k e^{-\theta_k}) - e^{-\theta_i} (- e^{-\theta_j})}{\sum_k e^{-\theta_k}} \\ 
&= 1\{y_i = 1\} -y_i \frac{1\{i = j \} \sum_k e^{-\theta_k} - e^{-\theta_j}}{\sum_k e^{-\theta_k}} \\ 
&= 1\{y_i = 1\} \frac{e^{-\theta_j} - 1\{i = j \} \sum_k e^{-\theta_k}}{\sum_k e^{-\theta_k}} \\ 
&= 1\{y_i = 1\} (\hat y_j  - 1\{i = j \}) \\ 
\end{align*}

Therefore, 

\begin{align*}
\frac{\partial J}{\partial \theta} = \begin{bmatrix}
\frac{\partial J}{\partial \theta_1} \\ 
\vdots \\
\frac{\partial J}{\partial \theta_i} \\ 
\vdots \\
\frac{\partial J}{\partial \theta_q} \\ 
\end{bmatrix} = \begin{bmatrix}
\hat y_1  - 0 \\ 
\vdots \\
\hat y_i - 1 \{y_i = 1 \} \\
\vdots \\
\hat y_q - 0\\
\end{bmatrix} = \hat y - y
\end{align*}

Neat result!

# (c)

Given

$$ h = sigmoid(z_1) = s(z_1) = s(x W_1 + b_1) $$
$$ \hat y = softmax(z_2) = f(z_2) = f(hW_2 + b_2)$$
$$ J = CE(y, \hat y) $$

So $W_1$ is of shape $q_0 \times q_1$, and $W_2$ is of shape $q_1 \times q_2$.

It appears that using **row vector** makes the math more easy and consistent:

\begin{align*}
\frac{\partial J}{\partial x} = \frac{\partial J}{\partial \hat y} \frac{\partial \hat y}{\partial z_2} \frac{\partial z_2}{\partial h} \frac{\partial h}{\partial z_1} \frac{\partial z_1}{\partial x}
\end{align*}

The corresponding matrix shape transformation is 

$$
1 \times q_0 = (1 \times q_2) (q_2 \times q_2) (q_2 \times q_1) (q_1 \times q_1) (q_1 \times q_0)
$$

When $q_i$ is the number of neurons in layer $i$ with $q_0$ meaning the number of neurons in the input layer.

This doesn't seem to be the case when $x$ is a **column vector**, which would be:

$$q_0 \times 1 \ne (q_2 \times 1) (q_2 \times q_2) (q_1 \times q_2) (q_1 \times q_1) (n \times q_1)$$

or maybe my linear algebra is wrong, please let me know if you know the right way to generalize to column vectors.

**Update (2017-10-24)**: see discussion on https://math.stackexchange.com/questions/2486150/jacobian-and-chain-rule. Indeed, row vector is more natural, but using colum vector is mostly about adding proper transpose operation.

Now, to solve $\frac{\partial J}{\partial x}$,

Shape: $1 \times q_2$,
$$
\frac{\partial J}{\partial \hat y} \frac{\partial \hat y}{\partial z_2} = \hat y - y
$$

Shape: $q_2 \times q_1$,
$$
\frac{\partial z_2}{\partial h} = W_2^T
$$

Shape: $q_1 \times q_1$ (the second equality HASN'T been proved yet),

$$\frac{\partial h}{\partial z_1} = \sigma '(z_1) = h(z_1)(1 - h(z_1))^T$$

Shape: $q_1 \times q_0$,

$$ \frac{\partial z_1}{\partial x} = W_1^T$$

Therefore, 

\begin{align*}
\delta_1 &= \frac{\partial J}{\partial z_2} = \hat y - y \\
\delta_2 &= \frac{\partial J}{\partial h}  = (\hat y - y)W_2^T \\
\delta_3 &= \frac{\partial J}{\partial z_1} = (\hat y - y) W_2^T \sigma'(z_1)\\
\frac{\partial J}{\partial x} &= (\hat y - y) W_2^T \sigma'(z_1) W_1^T
\end{align*}

**Question:** what does the empty circle ($\circ$) mean in $\delta_3 = \delta_2 \circ \sigma'(z_1)$ mean? In the solution file?

# (d)

There are $(D_x H + H) + (H D_y + D_y)$ parameters.

# (e), (f), (g)

These are programming exercises.